# Hands-On Assignment 5

In this assignment, you will practice everything that you have learned so far in an end-to-end setting.
You will be provided with a dataset that is **unique to you**, and your task is to perform
all the steps from previous assignments to clean, explore, visualize, and analyze your dataset.

**Written Portion**: Additionally, you will create a report that describes your process and provides insights about your dataset.
Each section that should appear in your report is noted with an orange star (like normal HO tasks).  The report should be  4-6 pages (12 pt font, 1.5 line spacing), and turned in on Canvas as a PDF.

The coding aspect for this assignment will be turned in the same was as all other HO's,
by submitting this file to the autograder.


For this assignment, feel free to make additional functions instead of implementing everything in the provided function.

The objective of this assignment is for you to apply and solidify the skills you have learned in previous assignments.

# Prompt

You have graduated from this class, and are a huge success!
You landed a job doing data science at some fancy company.

You just got a new client with some really interesting problems you get to solve.
Unfortunately, because of a big mess-up on their side the data's metadata got corrupted up
(and the person that used to maintain the data just took a vow of silence and moved to a bog).

The only column you are sure about is the `label` column,
which contains a numeric label for each row.
Aside from that, the client does not know anything about the names, content, or even data types for each column.

Your task is to explore, clean, and analyze this data.
You should have already received an email with the details on obtaining your unique data.
Place it in the same directory as this notebook (and your `local_grader.py` script) and name it `data.txt`.

*I know this prompt may sound unrealistic, but I have literally been in a situation exactly like this.
I was working at a database startup, and one of our clients gave us data with over 70 columns and more than a million records and told us:
"The person who used to manage the data is no longer working with us, but this was the data they used to make all their decisions.
We also lost all the metadata information, like column names."
...
Working in industry is not always glamorous.
-Eriq*

# Part 0: Explore Your Data

Before you start doing things to/with your data, it's always a good idea to load up your data and take a look.

In [1]:
import pandas
import numpy as np
import re
import sklearn.tree
import sklearn.neighbors
import sklearn.linear_model
from sklearn.model_selection import cross_validate
from scipy import stats

# Modify this to point to your data.
unique_data = pandas.read_csv('data.txt', sep = "\t")
unique_data

,label,col_00,col_01,col_02,col_03,col_04,col_05,col_06,col_07,col_08,col_09,col_10,col_11,col_12
0,5,908 N,0.6205,HUMAN COMPUTER INTERACTION,BIOTECHNOLOGY,779,0.9405,666,0.6983,andrew,0.8229,0.4855,235,0.8494
1,2,250 N,0.638,Statistics,Bioinformatics,216,0.4684,369,0.364,Michael,0.0561,0.5763,948,0.2661
2,3,69 N,-0.043,Bioengineering,Statistics,560,0.2707,865,0.1724,Fabrice,1.1116,0.5456,326,0.2838
3,1,733 N,0.9441,Computational Media,"Robotics Engineering, Electrical Engineering",408,0.4912,230,0.6314,Eriq,0.5697,0.4427,740,0.9536
4,4,152 N,0.9193,Natural Language Processing,Statistics,878,0.1301,415,0.1871,Tony,0.2189,0.2562,675,0.0621
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1268,0,794 N,0.3895,Electrical Engineering,"Computer Game Design, Computer Science",-147,0.8301,772,0.6212,Chris,0.4914,0.9488,3,0.1863
1269,2,497 N,0.6074,NaN,Computer Engineering,469,0.8746,79,0.6655,Michael,0.3519,0.167,898,0.5314
1270,0,659 N,0.2824,Electrical Engineering,Computer Science,118,0.7977,711,NaN,Chris,0.6803,1.021,63,0.2024
1271,0,858 N,0.1827,Data Science,Computer Game Design,97,0.7384,553,0.6537,Chris,0.2303,0.9324,220,0.0894


Don't forget to checkout the column information.

In [2]:
unique_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1273 entries, 0 to 1272
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   1273 non-null   int64 
 1   col_00  1261 non-null   object
 2   col_01  1261 non-null   object
 3   col_02  1267 non-null   object
 4   col_03  1259 non-null   object
 5   col_04  1255 non-null   object
 6   col_05  1264 non-null   object
 7   col_06  1265 non-null   object
 8   col_07  1262 non-null   object
 9   col_08  1262 non-null   object
 10  col_09  1265 non-null   object
 11  col_10  1266 non-null   object
 12  col_11  1259 non-null   object
 13  col_12  1258 non-null   object
dtypes: int64(1), object(13)
memory usage: 139.4+ KB


And any numeric information.

In [3]:
unique_data.describe()

,label
count,1273.00000
mean,2.51846
std,1.70548
min,0.00000
25%,1.00000
50%,3.00000
75%,4.00000
max,5.00000


<h4 style="color: darkorange; font-size: x-large";>★ Written Task: Introduction</h4>

Briefly describe the dataset you’re given and define the goal of the project and how you approach it.
For example, you can present a basic introduction of your data (shape and proposed data types)
and your goal is to use these features to predict the label of the response variable.
Then you propose a few models that are suitable for this project which will be introduced in the modeling section.

# Part 1: Data Cleaning

As always, we should start with data cleaning.
Take what you learned from HO3 to clean up this messy data to a point where it is ready for machine learning algorithms.

Some things you may want to do:
 - Deal with missing/empty values.
 - Fix numeric columns so that they actually contain numbers.
 - Remove inconsistencies from columns.
 - Assign a data type to each column.

<h4 style="color: darkorange; font-size: x-large";>★ Task 1.A</h4>

Complete the following function that takes in a DataFrame and outputs a clean version of the DataFrame.
You can assume that the frame has all the same structure as your unique dataset.
You can return the same or a new data frame.

In [4]:
def fix_empty_vals(values, type, replace):
    for i in range(len(values)):
        if not isinstance(values[i], type):
            values[i] = replace
    return values

def get_num(value):
    value = str(value)
    splitted = value.split()
    for val in splitted:
        if re.match('^\d+$', val):
            return (int(val), 0)
        elif re.match('^\d+\.\d+$', val):
            return (float(val), 1)
    return (str(value).lower(), 2)

def one_hot(frame, column_name):
    new_frame = frame.copy()
    d = {}
    if (len(column_name) == 0):
        return new_frame
    for i in range(len(frame[column_name])):
        val = str(frame[column_name][i])
        exports = val.split(",")
        for export in exports:
            export = export.strip().lower()
            if (column_name + ": " + export) not in d:
                d[column_name + ": " + export] = [0] * len(frame)
            d[column_name + ": " + export][i] = 1
    new_frame.pop(column_name)
    new_frame = pandas.concat([new_frame, pandas.DataFrame(d)], axis=1)
    return new_frame

def clean_data(frame):
    cp = frame.copy()
    for name in frame:
        if name == 'label':
            continue
        possible_types = [0, 0, 0] # int then float then str
        new_vals = [None] * len(frame[name])
        for i in range(len(frame[name])):
            new, type = get_num(frame[name][i])
            possible_types[type] += 1
            new_vals[i] = new
        cp[name] = new_vals
        col_type = possible_types.index(max(possible_types))
        if col_type == 0:
            cp[name] = fix_empty_vals(cp[name].values, int, 0)
            cp[name] = cp[name].astype(int)
        elif col_type == 1:
            cp[name] = fix_empty_vals(cp[name].values, float, 0.0)
            cp[name] = cp[name].astype(float)
    for name in cp:
        if cp[name].dtype != 'object' or name == 'col_08':
            continue
        cp = one_hot(cp, name)
    return cp

unique_data = clean_data(unique_data)
unique_data

,label,col_00,col_01,col_04,col_05,col_06,col_07,col_08,col_09,col_10,...,col_03: bioengineering,col_03: computer science,col_03: human computer interaction,col_03: applied mathematics,col_03: natural language processing,col_03: technology and information management,col_03: nan,col_03: data science,col_03: games and playable media,col_03: computer game design
0,5,908,0.6205,779,0.9405,666,0.6983,andrew,0.8229,0.4855,...,0,0,0,0,0,0,0,0,0,0
1,2,250,0.6380,216,0.4684,369,0.3640,michael,0.0561,0.5763,...,0,0,0,0,0,0,0,0,0,0
2,3,69,0.0000,560,0.2707,865,0.1724,fabrice,1.1116,0.5456,...,0,0,0,0,0,0,0,0,0,0
3,1,733,0.9441,408,0.4912,230,0.6314,eriq,0.5697,0.4427,...,0,0,0,0,0,0,0,0,0,0
4,4,152,0.9193,878,0.1301,415,0.1871,tony,0.2189,0.2562,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1268,0,794,0.3895,0,0.8301,772,0.6212,chris,0.4914,0.9488,...,0,1,0,0,0,0,0,0,0,1
1269,2,497,0.6074,469,0.8746,79,0.6655,michael,0.3519,0.1670,...,0,0,0,0,0,0,0,0,0,0
1270,0,659,0.2824,118,0.7977,711,0.0000,chris,0.6803,1.0210,...,0,1,0,0,0,0,0,0,0,0
1271,0,858,0.1827,97,0.7384,553,0.6537,chris,0.2303,0.9324,...,0,0,0,0,0,0,0,0,0,1


Now we should also be able to view all the numeric columns.

In [5]:
unique_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1273 entries, 0 to 1272
Data columns (total 47 columns):
 #   Column                                         Non-Null Count  Dtype  
---  ------                                         --------------  -----  
 0   label                                          1273 non-null   int64  
 1   col_00                                         1273 non-null   int64  
 2   col_01                                         1273 non-null   float64
 3   col_04                                         1273 non-null   int64  
 4   col_05                                         1273 non-null   float64
 5   col_06                                         1273 non-null   int64  
 6   col_07                                         1273 non-null   float64
 7   col_08                                         1273 non-null   object 
 8   col_09                                         1273 non-null   float64
 9   col_10                                         1273 

<h4 style="color: darkorange; font-size: x-large";>★ Written Task: Data Cleaning</h4>

Describe the steps you took for data cleaning.
Why did you do this?
Did you have to make some choices along the way? If so, describe them.

# Part 2: Data Visualization

Once you have cleaned up the data, it is time to explore it and find interesting things.
Part of this exploration, will be visualizing the data in a way that makes it easier for yourself and others to understand.
Use what you have learned in HO1 and HO2 to create some visualizations for your dataset.

In [35]:
# unique_data[unique_data['label'] == 1]['col_02: applied mathematics'].hist(bins=3)
start = unique_data.columns.get_loc('col_02: human computer interaction')
end = unique_data.columns.get_loc('col_02: applied mathematics')
for index, row in unique_data.iterrows():
    print(row)
#for i in range(start, end + 1):
        
    
    

label                                                 5
col_00                                              908
col_01                                           0.6205
col_04                                              779
col_05                                           0.9405
col_06                                              666
col_07                                           0.6983
col_08                                           andrew
col_09                                           0.8229
col_10                                           0.4855
col_11                                              235
col_12                                           0.8494
col_02: human computer interaction                    1
col_02: statistics                                    0
col_02: bioengineering                                0
col_02: computational media                           0
col_02: natural language processing                   0
col_02: electrical engineering                  

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



<h4 style="color: darkorange; font-size: x-large";>★ Written Task: Data Visualization</h4>

Create at least two different visualizations that help describe what you see in your dataset.
Include these visualizations in your report along with descriptions of
how you created the visualization,
what data preparation you had to do for the visualization (aside from the data cleaning in the previous part),
and what the visualization tells us about the data.

# Part 3: Modeling

Now that you have a good grasp of your clean data,
it is time to do some machine learning!
(Technically all our previous steps were also machine learning,
but now we get to use classifiers!)

Use the skills you developed to select **three** classifiers and implement them on your data.
For example, you can narrow down your choices to three classifiers which may include:
- Logistic regression
- K-nearest neighbors
- Decision tree
- Or others

<h4 style="color: darkorange; font-size: x-large";>★ Task 3.A</h4>

Complete the following function that takes in no parameters,
and returns a list with **three** untrained classifiers you are going to explore in this assignment.
This method may set parameters/options for the classifiers, but should not do any training/fitting.

For example, if you wanted to use logistic regression,
then **one** of your list items may be:
```
sklearn.linear_model.LogisticRegression()
```

In [7]:
def create_classifiers():
    clas_1 = sklearn.linear_model.LogisticRegression()
    clas_2 = sklearn.tree.DecisionTreeClassifier(max_depth = 3, random_state = 0)
    clas_3 = sklearn.neighbors.KNeighborsClassifier(n_neighbors = 3)
    return [clas_1, clas_2, clas_3]

my_classifiers = create_classifiers()
my_classifiers

[LogisticRegression(),
 DecisionTreeClassifier(max_depth=3, random_state=0),
 KNeighborsClassifier(n_neighbors=3)]

Now that we have some classifiers, we can see how they perform.

<h4 style="color: darkorange; font-size: x-large";>★ Task 3.B</h4>

Complete the following function that takes in an untrained classifier, a DataFrame, and a number of folds.
This function should run k-fold cross validation with the classifier and the data,
and return a list with the accuracy of each run of cross validation.
You can assume that the frame has the column `label` and the rest of the columns can be considered clean numeric features.

Note that you may have to break your frame into features and labels to do this.
Do not change the passed-in frame (make copies instead).

If you are getting any `ConvergenceWarning`s you may either ignore them,
or try and address them
(they will not affect your autograder score, but may be something to discuss in the written portion of this assignment).

In [8]:
def cross_fold_validation(classifier, frame, folds):
    features = frame.copy()
    for name in features:
        if features[name].dtype == 'object':
            features.pop(name)
    labels = features.pop('label')
    scores = cross_validate(classifier, features, labels, cv=5)
    return scores['test_score']

my_classifiers_scores = []
for classifier in my_classifiers:
    accuracy_scores = cross_fold_validation(classifier, unique_data, 5)
    my_classifiers_scores.append(accuracy_scores)
    print("Classifier: %s, Accuracy: %s." % (type(classifier).__name__, accuracy_scores))

/home/mnelhams/mnelhams/40/cse40_venv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/mnelhams/mnelhams/40/cse40_venv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules

Classifier: LogisticRegression, Accuracy: [0.94117647 0.96862745 0.96862745 0.98425197 0.95669291].
Classifier: DecisionTreeClassifier, Accuracy: [0.8745098  0.91764706 0.9254902  0.90551181 0.87795276].
Classifier: KNeighborsClassifier, Accuracy: [0.9372549  0.95294118 0.97647059 0.95669291 0.96062992].


<h4 style="color: darkorange; font-size: x-large";>★ Task 3.C</h4>

Complete the following function that takes in two equally-sized lists of numbers and a p-value.
This function should compute whether there is a statistical significance between
these two lists of numbers using a [Student's t-test](https://en.wikipedia.org/wiki/Student%27s_t-test)
at the given p-value.
Return `True` if there is a statistical significance, and `False` otherwise.
Hint: If you wish, you may use the `ttest_ind()` [method](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html) provided in the scipy package. 

In [9]:
def significance_test(a_values, b_values, p_value):
    result = stats.ttest_ind(a_values, b_values)
    return result.pvalue < p_value

for i in range(len(my_classifiers)):
    for j in range(i + 1, len(my_classifiers)):
        significant = significance_test(my_classifiers_scores[i], my_classifiers_scores[j], 0.10)
        print("%s vs %s: %s" % (type(my_classifiers[i]).__name__,
                                type(my_classifiers[j]).__name__, significant))

LogisticRegression vs DecisionTreeClassifier: True
LogisticRegression vs KNeighborsClassifier: False
DecisionTreeClassifier vs KNeighborsClassifier: True


<h4 style="color: darkorange; font-size: x-large";>★ Written Task: Modeling</h4>

Describe the classifiers you have chosen.
Be sure to include all details about any parameter settings used for the algorithms.

Compare the performance of your models using k-fold validation.
You may look at accuracy, F1 or other measures.

Then, briefly summarize your results.
Are your results statistically significant?
Is there a clear winner?
What do the standard deviations look like, and what do they tell us about the different models?
Include a table like Table 1.

<center>Table 1: Every table need a caption.</center>

| Model | Mean Accuracy | Standard Deviation of Accuracy |
|-------|---------------|--------------------------------|
| Logistic Regression | 0.724 | 0.004
| K-Nearest Neighbor | 0.750 | 0.003
| Decision Tree | 0.655 | 0.011

# Part 4: Analysis

Now, take some time to go over your results for each classifier and try to make sense of them.
 - Why do some classifiers work better than others?
 - Would another evaluation metric work better than vanilla accuracy?
 - Is there still a problem in the data that should fixed in data cleaning?
 - Does the statistical significance between the different classifiers make sense?
 - Are there parameters for the classifier that I can tweak to get better performance?

<h4 style="color: darkorange; font-size: x-large";>★ Written Task: Analysis</h4>

Discuss your observations, the relationship you found, and how you applied concepts from the class to this project.
For example, you may find that some feature has the most impact in predicting your response variable or removing a feature improves the model accuracy.
Or you may observe that your training accuracy is much higher than your test accuracy and you may want to explain what issues may arise.

# Part 5: Conclusion

<h4 style="color: darkorange; font-size: x-large";>★ Written Task: Conclusion</h4>

Briefly summarize the important results and conclusions presented in the project.
What are the important points illustrated by your work?
Are there any areas for further investigation or improvement?

<h4 style="color: darkorange; font-size: x-large";>★ Written Task: References</h4>

Include a standard bibliography with citations referring to techniques or published papers you used throughout your report (if you used any).

For example:
```
[1] Derpanopoulos, G. (n.d.). Bayesian Model Checking & Comparison.
https://georgederpa.github.io/teaching/modelChecking.html.
```

# Part XC: Extra Credit

So far you have used a synthetic dataset that was created just for you.
But, data science is always more interesting when you are dealing with actual data from the real world.
Therefore, you will have an opportunity for extra credit on this assignment using real-world data.

For extra credit, repeat the **written tasks** of Parts 0 through 4 with an additional dataset that you find yourself.
For the written portion of the extra credit for Part 0, include information about where you got the data and what the data represents.
You may choose any dataset that represents real data (i.e., is **not** synthetic or generated)
and is **not** [pre-packaged in scikit-learn](https://scikit-learn.org/stable/datasets.html).

Below are some of the many places you can start looking for datasets:
 - [Kaggle](https://www.kaggle.com/datasets) -- Kaggle is a website focused around machine learning competitions,
       where people compete to see who can get the best results on a dataset.
       It is very popular in the machine learning community and has thousands of datasets with descriptions.
       Make sure to read the dataset's description, as Kaggle also has synthetic datasets.
 - [data.gov](https://data.gov/) -- A portal for data from the US government.
        The US government has a lot of data, and much of it has to be available to the public by law.
        This portal contains some of the more organized data from several different government agencies.
        In general, the government has A LOT of interesting data.
        It may not always be clean (remember the CIA factbook), but it is interesting and available.
        All data here should be real-world, but make sure to read the description to verify.
 - [UCI's Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets.php) -- UC Irvine has their own data repository with a few hundred datasets on many different topics.
        Make sure to read the dataset's description, as UCI also has synthetic datasets.
 - [WHO's Global Health Observatory](https://apps.who.int/gho/data/node.home) -- The World Health Organization keeps track of many different health-related statistics for most of the countries in the world.
        All data here should be real-world, but make sure to read the description to verify.
 - [Google's Dataset Search](https://datasetsearch.research.google.com/) -- Google indexes many datasets that can be searched here.

You can even create a dataset from scratch if you find some data you like that is not already organized into a specific dataset.
The only real distinction between "data" and a "dataset" is that a dataset is organized and finite (has a fixed size).

Create a new section in your written report for this extra credit and include all the written tasks for the extra credit there.
Each written task/section that you complete for your new dataset is eligible for extra credit (so you can still receive some extra credit even if you do not complete all parts).
There is no need to submit any code for the extra credit.
If you created a new dataset, include the dataset or links to it with your submission.